In [1]:
import numpy as np
import cv2
import imutils
import pytesseract
from skimage.segmentation import clear_border
# to compute angles
# pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files (x86)\Tesseract-OCR\tesseract.exe'

In [ ]:
import time
import math

cap = cv2.VideoCapture('video.MOV')

frameRate = cap.get(5) #frame rate

x=1
while(cap.isOpened()):
    frameId = cap.get(1) #current frame number
    ret, frame = cap.read()
    if (ret != True):
        break
    if (frameId % math.floor(frameRate) == 0):
        filename = './test_images/image' +  str(int(x)) + ".jpg";x+=1
        cv2.imwrite(filename, frame)
cap.release()

In [2]:
# img = cv2.imread("test_images/image5.jpg")
# cv2.imshow('Cropped',img)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

In [3]:
def tratamento_imagem_deteccao_ret(img):
    grey = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # determinar as partes mais escuras com background claro
    rectKern = cv2.getStructuringElement(cv2.MORPH_RECT, (13, 5))
    blackhat = cv2.morphologyEx(grey, cv2.MORPH_BLACKHAT, rectKern) #blackhat morphological operation to reveal dark characters

    # fecha onde ha poucas partes escuras para encontrar a parte clara maior
    squareKern = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
    light = cv2.morphologyEx(grey, cv2.MORPH_CLOSE, squareKern)
    light = cv2.threshold(light, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]

    # compute the Scharr gradient representation of the blackhat
    # image in the x-direction and then scale the result back to
    # the range [0, 255]
    gradX = cv2.Sobel(blackhat, ddepth=cv2.CV_32F, dx=1, dy=0, ksize=-1)
    gradX = np.absolute(gradX)
    (minVal, maxVal) = (np.min(gradX), np.max(gradX))
    gradX = 255 * ((gradX - minVal) / (maxVal - minVal))
    gradX = gradX.astype("uint8")
    
    # blur the gradient representation, applying a closing
    # operation, and threshold the image using Otsu's method
    gradX = cv2.GaussianBlur(gradX, (5, 5), 0)
    gradX = cv2.morphologyEx(gradX, cv2.MORPH_CLOSE, rectKern)
    thresh = cv2.threshold(gradX, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]
    
    # perform a series of erosions and dilations to clean up the
    # thresholded image
    thresh = cv2.erode(thresh, None, iterations=2)
    thresh = cv2.dilate(thresh, None, iterations=2)
    
    # take the bitwise AND between the threshold result and the
    # light regions of the image
    thresh = cv2.bitwise_and(thresh, thresh, mask=light)
    thresh = cv2.dilate(thresh, None, iterations=2)
    thresh = cv2.erode(thresh, None, iterations=1)

    cnts = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = imutils.grab_contours(cnts)
    cnts = sorted(cnts, key=cv2.contourArea, reverse=True)[:5] #We’ll only return up to this many sorted license plate candidate contours.
    
    
    return cnts, grey

In [4]:
def license_detection(cnts, grey):
    # initialize the license plate contour and ROI
    lpCnt = None
    roi = None
    minAR = 4
    maxAR = 6
    clearBorder = False
    # loop over the license plate candidate contours
    ar_list = []
    lp = None
    for c in cnts:
        # compute the bounding box of the contour and then use
        # the bounding box to derive the aspect ratio
        (x, y, w, h) = cv2.boundingRect(c)
        ar = w / float(h)
        if ar >= minAR and ar <= maxAR:
        # store the license plate contour and extract the
        # license plate from the grayscale image and then
        # threshold it
            lpCnt = c
            lp = grey[y:y + h, x:x + w]
            roi = cv2.threshold(lp, 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]
            # check to see if we should clear any foreground
            # pixels touching the border of the image
            # (which typically, not but always, indicates noise)
            if clearBorder:
                roi = clear_border(roi)
            # display any debugging information and then break
            # from the loop early since we have found the license
            # plate region
    #         cv2.imshow('Cropped',licensePlate)
    #         cv2.waitKey(0)
    #         cv2.destroyAllWindows()
    #         cv2.imshow('ROI',roi)
    #         cv2.waitKey(0)
    #         cv2.destroyAllWindows()
            break
    
    return roi, lp


In [5]:
# tell Tesseract to only OCR alphanumeric characters
alphanumeric = "ABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789"
options = "-c tessedit_char_whitelist={}".format(alphanumeric)
# set the PSM mode
options += " --psm {}".format(7)
# return the built options string

In [6]:
lp_test_list = []
for i in range(29):
    img = cv2.imread(f"test_images/image{i+1}.jpg")
    
    cnts, grey = tratamento_imagem_deteccao_ret(img)
    roi, licensePlate = license_detection(cnts, grey)
    
    
    # initialize the license plate text
    lpText = None
    if roi is not None:
        # OCR the license plate
        lpText = pytesseract.image_to_string(licensePlate, config=options)
        if lpText not in lp_test_list:
            lp_test_list.append(lpText)
            print(str(lpText))
            filename = './resultado/placa' +  str(lpText) + ".jpg"
            cv2.imwrite(filename, licensePlate)
            
#             filename = './test_images/image' +  str(int(x)) + ".jpg";x+=1
#             cv2.imwrite(filename, frame)
#         cv2.imshow(lpText,licensePlate)
#         cv2.waitKey(0)
#         cv2.destroyAllWindows()
        
    
    # return a 2-tuple of the OCR'd license plate text along with
    # the contour associated with the license plate region
#     filename = './resultado/' +  str(lpText) + ".jpg";x+=1
#     cv2.imwrite(filename, frame)



29A33185

RYEE

30A61235

199490101

99A490101

30A61329

30V4495



In [7]:
# import time
# import math

# cap = cv2.VideoCapture('Traffic_Camera.avi')

# frameRate = cap.get(5) #frame rate

# x=1
# while(cap.isOpened()):
#     frameId = cap.get(1) #current frame number
#     ret, frame = cap.read()
#     if (ret != True):
#         break
#     if (frameId % math.floor(frameRate) == 0):
#         filename = './test_images/image' +  str(int(x)) + ".jpg";x+=1
#         cv2.imwrite(filename, frame)
# cap.release()